In [1]:
import pickle
import pandas as pd
import numpy as np
import nrrd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output

from radiomics import featureextractor, getTestCase
import SimpleITK as sitk
import subprocess

import os
import glob
import shutil

import time
import datetime

print(datetime.datetime.now())

2022-02-22 18:43:41.890304


In [2]:
path_data = '../data/d20220201/'

# INPUTS
fn_patient = '../metadata/Celine_FINAL_RADIOPOP_DATA_with LRE Death_220126.xlsx'
fn_patient2 = '../metadata/List Celine Radipop 20.12.21.xlsx'
path_masks = path_data + '0raw_masks/'
path_pngs = path_data + '0raw_png/'
path_nrrdmeta = path_data + 'nrrds_metadata/'

# OUTPUTS
path_nrrds = path_data + 'nrrds_matched2mask/'

In [3]:
# prepare directories
try:
    os.mkdir(path_nrrds)
except OSError:
    print ("Creation of the directory %s failed" % path_nrrds)
else:
    print ("Successfully created the directory %s " % path_nrrds)

Creation of the directory data/d20220201/nrrds_matched2mask/ failed


In [4]:
patients_df = pd.read_excel(fn_patient, index_col=0, dtype=str)#, index_col=0, header=1, nrows=371, dtype=str)
patients_df['patientID'] = patients_df.index
patients_df['patientID'] = patients_df['patientID'].apply(lambda x: str(x).zfill(4))
patients_df = patients_df.set_index('patientID')
patients_df

patients_df2 = pd.read_excel(fn_patient2, index_col=0, dtype=str)#, index_col=0, header=1, nrows=371, dtype=str)
patients_df2['temp'] = patients_df2.index
patients_df2['patientID'] = patients_df2['temp'].str.split(" ", n=1).str[0]
patients_df2.patientID.fillna(patients_df2.temp, inplace=True)
patients_df2 = patients_df2[patients_df2['patientID'].astype(int) >= 1000]
patients_df2 = patients_df2.set_index('patientID')
patients_df2

listPatients = patients_df.index.tolist() + patients_df2.index.tolist()
print(len(listPatients))

562


In [5]:
t0 = time.time()

#listMasks = [x for x in os.listdir(path_masks) if os.path.isdir(os.path.join(path_masks, x))]
listConvSuccess = []
listNoImgs = []
listConvError = []
png2npError = []
metaGrabError = []
np2nrrdError = []

# for each available mask, check for png image file
for eaMask in listPatients:
    patientID = eaMask
    dir_png = path_pngs + str(patientID)
    if os.path.exists(dir_png):
        path_nrrdTarget = path_nrrds + str(patientID) + '.nrrd'
        if os.path.exists(path_nrrdTarget):
            print('ID_' + str(patientID) + ': nrrd already complete')
            listConvSuccess.append(patientID)
        else:
            try:
                t1 = time.time()
                # convert png to numpy array
                fns_pngs = np.array([x for x in os.listdir(dir_png) if x.endswith(".png") and not x.startswith("._")])
                i_pngs = np.array([x.replace(".png", "") for x in fns_pngs], dtype = int)
                image = np.array([plt.imread(os.path.join(dir_png, x)) for x in fns_pngs[np.argsort(i_pngs)]])
                image = image.transpose()
                try:    
                    # get the dicom meta data
                    meta_nrrd = nrrd.read(path_nrrdmeta + str(eaMask) + '.nrrd')
                    currMeta = meta_nrrd[1]
                    currMeta["type"] = "int"
                    try:
                        # convert numpy array to nrrd
                        nrrd.write(path_nrrdTarget, image, header=currMeta)

                        print('ID_' + str(patientID) + ': converted, ' + str(time.time() - t1) + ' seconds.')
                        listConvSuccess.append(eaMask)
                    except:
                        print('ID_' + str(patientID) + ': np to nrrd conversion error')
                        listConvError.append(eaMask)
                        np2nrrdError.append(eaMask)
                except:
                    print('ID_' + str(patientID) + ': dicom meta grab error')
                    listConvError.append(eaMask)
                    metaGrabError.append(eaMask)
            except:
                print('ID_' + str(patientID) + ': png to numpy conversion error')
                listConvError.append(eaMask)
                png2npError.append(eaMask)
    else:
        print('ID_' + str(eaMask) + ': image files missing')
        listNoImgs.append(eaMask)
    
print(time.time() - t0)
print(datetime.datetime.now())

ID_0001: nrrd already complete
ID_0002: nrrd already complete
ID_0004: nrrd already complete
ID_0005: nrrd already complete
ID_0006: nrrd already complete
ID_0007: nrrd already complete
ID_0008: nrrd already complete
ID_0009: nrrd already complete
ID_0010: nrrd already complete
ID_0011: nrrd already complete
ID_0013: nrrd already complete
ID_0015: nrrd already complete
ID_0016: nrrd already complete
ID_0018: nrrd already complete
ID_0019: nrrd already complete
ID_0020: nrrd already complete
ID_0021: nrrd already complete
ID_0022: nrrd already complete
ID_0023: nrrd already complete
ID_0024: nrrd already complete
ID_0025: nrrd already complete
ID_0026: nrrd already complete
ID_0027: nrrd already complete
ID_0028: nrrd already complete
ID_0029: nrrd already complete
ID_0030: nrrd already complete
ID_0031: nrrd already complete
ID_0032: nrrd already complete
ID_0033: nrrd already complete
ID_0034: nrrd already complete
ID_0035: nrrd already complete
ID_0036: nrrd already complete
ID_0037:

ID_0311: converted, 19.513123512268066 seconds.
ID_0312: nrrd already complete
ID_0313: nrrd already complete
ID_0314: nrrd already complete
ID_0317: nrrd already complete
ID_0318: nrrd already complete
ID_0319: nrrd already complete
ID_0321: nrrd already complete
ID_0322: nrrd already complete
ID_0323: nrrd already complete
ID_0324: nrrd already complete
ID_0325: nrrd already complete
ID_0326: nrrd already complete
ID_0327: nrrd already complete
ID_0330: nrrd already complete
ID_0331: nrrd already complete
ID_0332: nrrd already complete
ID_0333: nrrd already complete
ID_0334: nrrd already complete
ID_0335: nrrd already complete
ID_0336: nrrd already complete
ID_0337: nrrd already complete
ID_0338: nrrd already complete
ID_0339: nrrd already complete
ID_0340: nrrd already complete
ID_0341: nrrd already complete
ID_0342: nrrd already complete
ID_0343: nrrd already complete
ID_0346: nrrd already complete
ID_0347: nrrd already complete
ID_0349: nrrd already complete
ID_0350: nrrd already 

In [6]:
print('Problem with conversion')
print(listConvError)

print('png 2 np')
print(png2npError)
print('meta grab problem')
print(metaGrabError)
print('np 2 nrrd')
print(np2nrrdError)

print('No raw files:')
print(listNoImgs)

Problem with conversion
[]
png 2 np
[]
meta grab problem
[]
np 2 nrrd
[]
No raw files:
[]


In [7]:
# check which meta grab problems are missing meta files

setMetaGrab = set(metaGrabError)
metaAvail = set([x.split('.')[0] for x in os.listdir(path_nrrdmeta)])

print(sorted(list(setMetaGrab - setMetaGrab.intersection(metaAvail))))

path_dcmzips = path_data + '0raw_dicom/'
dicomAvail = set([x.split('.')[0] for x in os.listdir(path_dcmzips)])

print(sorted(list(setMetaGrab - setMetaGrab.intersection(dicomAvail))))

[]
[]


In [8]:
# removed 20201231: i think it's unnecessary

# t0 = time.time()

# listNrrd = [x for x in os.listdir(path_tempnrrds)]

# # for each available nrrd, transpose, update size metadata, and save
# for eaNrrd in listNrrd:
#     currFile = nrrd.read(path_tempnrrds + '/' + eaNrrd)
#     currImage = currFile[0].transpose()  # why??
#     currDict = currFile[1]
#     currDict['sizes'] = currImage.shape
#     # save
#     nrrd.write(path_nrrds + '/' + eaNrrd, currImage, header=currDict)

# print(time.time() - t0)
# print(datetime.datetime.now())

In [9]:
print(datetime.datetime.now())

2022-02-22 18:44:03.986986
